# Data Modelisation

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import gensim

In [ ]:
data = pd.read_csv("datasets/posts.clean.csv")
data.head(3)

## 1) Topic Modelling